In [1]:
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
hp = pd.Series(np.random.uniform(-0.05, 0.15, 36), index=pd.date_range('1/1/2011', periods=36, freq='M'))
hp.head()

2011-01-31    0.038007
2011-02-28    0.090947
2011-03-31    0.144175
2011-04-30   -0.037795
2011-05-31    0.008566
Freq: M, dtype: float64

In [6]:
start = datetime.datetime(2010, 12, 31)
end = datetime.datetime(2013, 12, 31)
spy = web.DataReader("SPY", 'yahoo', start, end)[['Adj Close']]
spy = spy.resample('M').last().pct_change()[1:]
spy.head()

,Adj Close
Date,
2011-01-31,0.023300
2011-02-28,0.034738
2011-03-31,0.000120
2011-04-30,0.028962
2011-05-31,-0.011215


In [10]:
gspc = web.DataReader("^GSPC", 'yahoo', start, end)[['Adj Close']]
gspc = gspc.resample('M').last().pct_change()[1:]
gspc.head()

,Adj Close
Date,
2011-01-31,0.022646
2011-02-28,0.031957
2011-03-31,-0.001047
2011-04-30,0.028495
2011-05-31,-0.013501


In [15]:
tbl = pd.concat([hp,spy,gspc], join='outer', axis = 1)
tbl.columns = ['port','altport','bench']
tbl.head()

,port,altport,bench
2011-01-31,0.038007,0.023300,0.022646
2011-02-28,0.090947,0.034738,0.031957
2011-03-31,0.144175,0.000120,-0.001047
2011-04-30,-0.037795,0.028962,0.028495
2011-05-31,0.008566,-0.011215,-0.013501


The output above is the same one as at the top of page 197.

In [17]:
rf = 0.0007

In [22]:
from __future__ import division
annual_port_ret = tbl['port'].mean()*12
annual_port_std = tbl['port'].std()*(12**0.5)
sharpe_port = (annual_port_ret - rf) / annual_port_std
print sharpe_port

2.40714819253


In [23]:
annual_altport_ret = tbl['altport'].mean()*12
annual_altport_std = tbl['altport'].std()*(12**0.5)
sharpe_altport = (annual_altport_ret - rf) / annual_altport_std
print sharpe_altport

1.28885276226


The output above is similar to the one on pages 197-199.

In [25]:
mar = 0.015

In [26]:
roy_port = (annual_port_ret - mar) / annual_port_std
print roy_port

2.33452561135


In [27]:
roy_altport = (annual_altport_ret - mar) / annual_altport_std
print roy_altport

1.17098214281


The output above is similar to the one on pages 199-200.

In [33]:
import statsmodels.formula.api as sm
result = sm.ols(formula="port ~ bench", data=tbl).fit()
port_beta = result.params[1]

In [36]:
treynor_port = (annual_port_ret - rf) / port_beta
print treynor_port

2.01118367287


In [35]:
result2 = sm.ols(formula="altport ~ bench", data=tbl).fit()
altport_beta = result2.params[1]

In [37]:
treynor_altport = (annual_altport_ret - rf) / altport_beta
print treynor_altport

0.156577484254


The output above is similar to the one on pages 201-202.

In [45]:
period_mar = mar / 12

In [46]:
dd_port = tbl[tbl['port']<period_mar]['port'].std()*(12**0.5)
sortino_port = (annual_port_ret - mar) / dd_port
print sortino_port

8.94245619453


In [47]:
dd_altport = tbl[tbl['altport']<period_mar]['altport'].std()*(12**0.5)
sortino_altport = (annual_altport_ret - mar) / dd_altport
print sortino_altport

1.75710815489


The output above is similar to the one on pages 203-204.

In [51]:
tbl['Act_Ret_port'] = tbl['port'] - tbl['bench']
alpha_port = tbl['Act_Ret_port'].mean()*12
tracking_error_port = tbl['Act_Ret_port'].std()*(12**0.5)
ir_port = alpha_port/tracking_error_port
print ir_port

1.56862715017


In [52]:
tbl['Act_Ret_altport'] = tbl['altport'] - tbl['bench']
alpha_altport = tbl['Act_Ret_altport'].mean()*12
tracking_error_altport = tbl['Act_Ret_altport'].std()*(12**0.5)
ir_altport = alpha_altport / tracking_error_altport
print ir_altport

6.03832212577


The output above is similar to the one on pages 205-206.

In [62]:
d = {'Portfolio' : [sharpe_port,roy_port,treynor_port,sortino_port,ir_port],'Alt Portfolio' : [sharpe_altport,roy_altport,treynor_altport,sortino_altport,ir_altport]}
data = pd.DataFrame(d, index=['Sharpe','Roy SF','Treynor','Sortino','Info Ratio'])
data.transpose().round(2)

,Sharpe,Roy SF,Treynor,Sortino,Info Ratio
Alt Portfolio,1.29,1.17,0.16,1.76,6.04
Portfolio,2.41,2.33,2.01,8.94,1.57


The output above is similar to the one on page 207.